# VGG16 Sample 100 Epochs

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 5GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# Constant Parameters
TRAIN_DIR = "../data/sample/"
MODEL_DIR = "../models/"
VALIDATION_SPLIT = 0.2
IMAGE_HEIGHT = IMAGE_WIDTH = 224
BATCH_SIZE = 64
SEED = 1

# Training Parameters
NUM_EPOCHS = 100
STARTING_EPOCH = 15
OPTIMIZER = keras.optimizers.Adam()
LOSS_FN = keras.losses.CategoricalCrossentropy()

In [4]:
model_id = "vgg16_8_100epochs"              # Unique Identifier used for saving model
color_mode = "rgb"                             # "rgb", "rgba" or "grayscale"
image_depth = 3

In [5]:
# Model Architecture
def get_model(image_depth, num_classes):
    model = keras.models.load_model(MODEL_DIR + "vgg16_8_15epochs.h5")
    return model

In [6]:
# Data Augmentation Parameters
image_generator = ImageDataGenerator(
    validation_split = VALIDATION_SPLIT,
    rescale = 1./255,
    rotation_range = 45,
    horizontal_flip = True,
    zoom_range = 0.4
)

In [7]:
train_data_gen = image_generator.flow_from_directory(
    color_mode = color_mode,
    directory = TRAIN_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle = True,
    seed = SEED,
    class_mode = "categorical",
    subset = "training"
)

Found 13440 images belonging to 42 classes.


In [8]:
validation_data_gen = image_generator.flow_from_directory(
    color_mode = color_mode,
    directory = TRAIN_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode = "categorical",
    subset = "validation"
)

Found 3360 images belonging to 42 classes.


In [9]:
model = get_model(image_depth, train_data_gen.num_classes)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 42)                43050     
Total params: 15,283,050
Trainable params: 568,362
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
history = model.fit(
    train_data_gen,
    epochs = NUM_EPOCHS,
    initial_epoch = STARTING_EPOCH,
    steps_per_epoch = train_data_gen.n // BATCH_SIZE,
    validation_data = validation_data_gen,
    callbacks = [
        keras.callbacks.CSVLogger(MODEL_DIR + "{}.csv".format(model_id), append = True),
        keras.callbacks.EarlyStopping(monitor = "val_accuracy",
                                    mode = "max",
                                    patience = 10,
                                    verbose = 1,
                                    restore_best_weights = True),
        keras.callbacks.ModelCheckpoint(MODEL_DIR + "checkpoints/" + model_id + ".{epoch:03d}-{val_accuracy:.4f}",
                                      verbose = 1,
                                      save_weights_only = False,
                                      save_freq = "epoch")
    ]
)

Epoch 16/100
210/210 [==============================] - ETA: 0s - loss: 1.9132 - accuracy: 0.4715
Epoch 00016: saving model to ../models/checkpoints/vgg16_8_100epochs.016-0.4390
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../models/checkpoints/vgg16_8_100epochs.016-0.4390\assets
210/210 [==============================] - 264s 1s/step - loss: 1.9132 - accuracy: 0.4715 - val_loss: 2.0553 - val_accuracy: 0.4390
Epoch 17/100
210/210 [==============================] - ETA: 0s - loss: 1.8950 - accuracy: 0.4798
Epoch 00017: saving model to ../models/checkpoints/vgg16_8_100epochs.017-0.4506
INFO:tensorflow:Assets written to: ../models/checkpoints/vgg16_8_100epochs.017-0.4506\assets
210/210 [==============================] - 257s 1s/step - loss: 1.8950 - accuracy: 0.4798 - val_loss: 2.0561 - val_accuracy: 0.4506
Epoch 18/100
210/210 [==============================] - ETA: 0s - loss: 1.8723 - accuracy: 0.4838
Epoch 00018: sa

INFO:tensorflow:Assets written to: ../models/checkpoints/vgg16_8_100epochs.035-0.4747\assets
210/210 [==============================] - 280s 1s/step - loss: 1.6633 - accuracy: 0.5333 - val_loss: 1.9798 - val_accuracy: 0.4747
Epoch 36/100
210/210 [==============================] - ETA: 0s - loss: 1.6589 - accuracy: 0.5400
Epoch 00036: saving model to ../models/checkpoints/vgg16_8_100epochs.036-0.4774
INFO:tensorflow:Assets written to: ../models/checkpoints/vgg16_8_100epochs.036-0.4774\assets
210/210 [==============================] - 282s 1s/step - loss: 1.6589 - accuracy: 0.5400 - val_loss: 1.9694 - val_accuracy: 0.4774
Epoch 37/100
210/210 [==============================] - ETA: 0s - loss: 1.6559 - accuracy: 0.5354
Epoch 00037: saving model to ../models/checkpoints/vgg16_8_100epochs.037-0.4753
INFO:tensorflow:Assets written to: ../models/checkpoints/vgg16_8_100epochs.037-0.4753\assets
210/210 [==============================] - 279s 1s/step - loss: 1.6559 - accuracy: 0.5354 - val_loss:

KeyboardInterrupt: 

In [ ]:
acc = history.history["accuracy"]
loss = history.history["loss"]
validation_acc = history.history["val_accuracy"]
validation_loss = history.history["val_loss"]

epochs_range = range(len(history.history["accuracy"]))

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, validation_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, validation_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
model.save(MODEL_DIR + "{}".format(model_id), overwrite = False)